# COGS 108 - How do keywords affect social media content popularity?

# Overview

Using DeviantArt API tools, we examine how certain keywords in content captions correlate with the popularity and visibility of artwork. We also look at the relationship between viewership, likes/favorites, comments, and "Author Watchers" to see how they may–or may not–affect the popularity of DeviantArt content with the captions. Lastly, we use several models, such as Decisions Trees, OLS, Lassos, and Linear Regressions, to further examine the significance of caption keywords and their influence in popularity.

# Names

- Mariam Bachar
- Alexandra Hernandez
- Brian Kwon 
- Andrew Uhm
- Ethan Wang 

<a id='research_question'></a>
# Research Question

Do certain keywords as identified by CLIP correlate with the popularity (as measured by the equivalent of “likes”) that artwork receives on social media?

<a id='background'></a>

## Background & Prior Work

With the rise of social media in the past decade, Internet users are constantly exposed to different types of media, especially digital art. To interact with them, users can  “like” and “comment” on another’s content, which also notifies the owner of their received feedback. In turn, the algorithm not only encourages users to continue seeing content from the original user, but also similar “liked” content from different users (Lua, 2023). As a result, social media algorithms–despite having the same code throughout–are customized to each user, showing them content that is relevant to them, based on their past interactions with other accounts and users. Furthermore, these processes incorporate machine learning algorithms, such as feature detection and sentiment classification to better pinpoint content and users that an individual will most likely interact with, boosting engagement for both the receiving user–and the social media itself (T.K. et al., 2021).

As social media continues to grow, there continues to be several cases done on the usage of social media algorithms and their role in user engagement. Especially with the introduction of new feature detection technologies–and Artificial Intelligence (AI)–the study field has developed exponentially. In Radion Purba et. al. 's “Instagram Post Popularity Trend Analysis and Prediction” (2020) they looked into social media popularity predictions for marketing purposes; particularly, their research methodology discussed provided a similar workflow to our posed research question. They also discussed definitions of popularity, derived from the data collected. However, the main difference is the study done using hashtags (Radion Purba et al., 2020); we would incorporate AI-generated descriptions of relevant artwork. Similarly, Kafritsas's “CLIP: The Most Influential AI Model from OpenAI and How To Use It” (2022)  details the AI software we plan to use for our data generation. It involves the  AI, CLIP, showcasing  examples of its function, and describes common use cases. CLIP is not necessarily as accurate as other supervised learning models and lacks some context interpreting ability. As we plan to use similar AI technologies in our research, Kafritsas’s usage of CLIP primes us for its use in a concise manner while also precautions us on making certain assumptions when making our analyses.

**References**
1. Lua, A. (2023, April 20). How the Instagram Algorithm Works in 2023: Everything You Need to Know. Buffer Library; Buffer Library. https://buffer.com/library/instagram-feed-algorithm/#how-does-the-instagram-algorithm-work-the-6-key-ranking-factors

2. T.K., B., Annavarapu, C. S. R., & Bablani, A. (2021). Machine learning algorithms for social media analysis: A survey. Computer Science Review, 100395. https://doi.org/10.1016/j.cosrev.2021.100395

3. Radion Purba, K., Asirvatham, D., & Kumar Murugesan, R. (2020). Instagram Post Popularity Trend Analysis and Prediction using Hashtag, Image Assessment, and User History Features. The International Arab Journal of Information Technology, 1. https://doi.org/10.34028/iajit/18/1/10

4. Kafritsas, N. (2022). CLIP: The Most Influential AI Model From OpenAI — And How To Use It. Medium; Towards Data Science. https://towardsdatascience.com/clip-the-most-influential-ai-model-from-openai-and-how-to-use-it-f8ee408958b1

# Hypothesis


We predict that digital artwork that contains certain keywords as predicted by CLIP (painting vs. watercolor vs. digital) will indeed have a positive correlation to popularity on social media. As humans observing what is popular, we notice that certain features tend to repeat themselves across posts, which leads us to believe a correlation will be found.

# Dataset(s)

- Dataset Name: deviation_info
- Link to the dataset: https://github.com/COGS108/Group_Sp23_Project_Group_3/blob/master/deviation_info.csv
- Number of observations: 1188

This dataset is a set of deviations (that is images from deviantart) that contain deviation ids and metadata about the deviation itself as well as the author. It does not include the actual images.

- Dataset Name: caption_info
- Link to the dataset: https://github.com/COGS108/Group_Sp23_Project_Group_3/blob/master/caption_info.csv
- Number of observations: 1188

This dataset is a set of captions processed from the image, corresponding to a deviation id. It was processed using the CLIP interrogator in Automatic1111's stable diffusion webui.

- Dataset Name: images
- Link to the dataset: https://github.com/COGS108/Group_Sp23_Project_Group_3/tree/master/images
- Number of observations: 1198

This dataset is a directory of images in png format that are named based on their corresponding deviation ids, it contains the actual images. There are 10 extra images in here that aren't found in our other datasets.

All of the datasets were built from scraping, and use deviation ids as their identifiers. Because of this, we can easily add them together based on those deviation ids if necessary.

# Setup

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import rankdata

import requests
import urllib
from bs4 import BeautifulSoup
import deviantart

import time
from datetime import datetime
from pathlib import Path

import json
from PIL import Image
import base64
import cv2

import nltk
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from scipy import stats
import math
import random
import statsmodels.api as sm
import patsy

from IPython.display import Image, display
import warnings
warnings.filterwarnings('ignore')

# DeviantArt API: https://www.deviantart.com/developers/http/v1/20210526
# Open-Source Python wrapper for DA API: https://github.com/neighbordog/deviantart

ModuleNotFoundError: No module named 'deviantart'

In [ ]:
# creates a pd df from the CSV file if it exists, else creates a blank df.
csv_file = 'deviation_info.csv'
try:
    deviation_df = pd.read_csv(csv_file)
except FileNotFoundError:
    deviation_df = pd.DataFrame()

# Data Cleaning

First, we'll do some basic cleaning where we drop N/A values and duplicates.

In [ ]:
# if new session, retrieve our deviation info
csv_file = 'deviation_info.csv'
try:
    deviation_df = pd.read_csv(csv_file)
except:
    FileNotFoundError
    
# if new session, retrieve our CLIP caption info
csv_file = 'caption_info.csv'
try:
    caption_df = pd.read_csv(csv_file)
except FileNotFoundError:
    caption_df = pd.DataFrame()

Let's join our dataframes together based on their common ID for easier usage later:

In [ ]:
# joining the two dataframes for good measure.
df = pd.merge(deviation_df, caption_df, on='Deviation ID', how='inner');
df.head();

We want to anonymize the data to maintain ethical integrity as discussed in our project proposal. Let's remove the Author field, as it doesn't contribute anything otherwise. We don't need the URL anymore and it also reveals the author on top of that, so we remove it too. We are essentially using the caption in place of the title, so we will disregard it. We also acknowledge that webpage "views" can be manipulated by bots. The views on the deviation (read: artwork) are still relevant despite knowing that, but the views on the Author's page matter less so. Let's remove that also:

In [ ]:
df = df.drop('URL Link', axis=1)
df = df.drop('Author', axis=1)
df = df.drop('Author Page Views', axis=1)
df = df.drop('Title', axis=1)

In [ ]:
df.head();

We can justify the rest of the columns. We need the ID for identification purposes. The title may relate to the CLIP caption. The views, favorites, and comments are how we are gauging the popularity of the post. The date posted is relevant for trend analysis. The file size, height, and width implies the rendering definition (e.g. 1080px) of the work. Author watchers and deviations may imply how much experience or traction this author has had on DeviantArt. Finally, the CLIP caption is integral to our hypothesis.

Now we consider adjusting the string-based content of our data. Below we notice that the AI caption is longer than we need it to be:

In [ ]:
for n in range(0, 1000, 200):
    print(df['Caption'][n]);
    print();

Notice that mainly what is before the first comma is the only relevant part of the caption. Furthermore, the AI makes a guess at who made the picture after the first comma. Let's remove everything after the first comma:

In [ ]:
for n in range(df['Caption'].size):
    before_comma = df['Caption'][n].split(",")[0]
    df.loc[n, 'Caption'] = before_comma

In [ ]:
df['Caption'][30]

Now the caption is much more precise. However, we do not want the stopwords such as "a", "is", "the", etc. within our analysis, as they are irrelevant. Let's remove them:

In [ ]:
stopwords = set(stopwords.words('english'))

In [ ]:
for n in range(df['Caption'].size):
    tokens = word_tokenize(df['Caption'][n])
    filtered_text = [word for word in tokens if word not in stopwords]
    if "'s" in filtered_text:
        filtered_text.remove("'s")
    filtered_text = list(set(filtered_text))
    filtered_caption = ' '.join(filtered_text)
    df.loc[n, 'Caption'] = filtered_caption

In [ ]:
df['Caption'][30]

This will do.

# Data Analysis & Results

The first step of our Data Analysis would be to determine the popularity metric by using views, favorites, and comments fields. This way, we can dive deeper into our analysis and explore 3 main popularity trends: fileSizeAndQuality, authorExperience, and the AICaption.

Next, we will run a statistical test to see whether popularity IS NOT related to fileSizeAndQuality or authorExperience, and if popularity IS related to the AICaption. It's not critical that these statistical tests pass, rather we just need the results to evaluate whether our hypothesis is true or false;

We predict that digital artwork that contains certain keywords as predicted by CLIP (painting vs. watercolor vs. digital) will indeed have a positive correlation to popularity on social media. As humans observing what is popular, we notice that certain features tend to repeat themselves across posts, which leads us to believe a correlation will be found.

We first need to define a popularity metric in order to do meaningful analysis. We explore the views, favorites, comments, and author watcher fields to get an idea of what a reasonable metric to measure popularity would look like.

In [ ]:
sns.histplot(df['Views']/100)
plt.xlim(0, 40000)
plt.show()

The plot shown above is a histogram of the artwork views divided by 100. The x-axis represents the range of views, while the y-axis represents the frequency or count of artworks falling within each range. The majority of artworks fall between 0 and 5000 views.

In terms of the research question and hypothesis, the histogram provides an overview of the distribution of artwork views on social media. By examining the histogram, we can analyze the relationship between the popularity of artwork (measured by views) and the keywords present in the artwork captions.

In [ ]:
sns.histplot(df['Favorites'])
plt.show()

The plot shown is a histogram of the number of favorites (equivalent to "likes") received by the artworks. The histogram allows us to examine the relationship between the popularity of the artworks (measured by the number of favorites) and the keywords present in their captions. The x-axis represents the range of favorites, while the y-axis represents the frequency or count of artworks falling within each range. As one can see, majority of art received favorites between 5000 to 15000.

By looking at the histogram, we can gain insights into how artworks are being received by the audience in terms of favorites. The distribution of favorites can indicate whether certain artworks are more popular or resonate better with the viewers.

In [ ]:
sns.histplot(df['Comments'])
plt.xlim(0, 10000)
plt.show()

The plot shown is a histogram of the number of comments received by the artworks. Similar to the previous plot, the x-axis represents the range of comments, while the y-axis represents the frequency or count of artworks falling within each range, with the majority falling between 0 and 1000 comments.

Analyzing this plot in relation to the research question and hypothesis, it provides an overview of the distribution of comments received by the artworks on social media. The histogram allows us to examine the relationship between the engagement level of the artworks (measured by the number of comments) and the keywords present in their captions.

We notice that the distribution for views, favorites, and comments are all similar but differ obviously on the scale. There are a handful of outliers here and there (two points at around 10 million and 6 million are excluded in the views plot and around 5 points above 10 thousand were excluded in the comments plot), but for the most part most people do not get a lot of interaction on their posts (which is how popularity intuitively works).

In [ ]:
sns.histplot(df['Author Watchers'])
plt.show()

Author distribution is certainly not normal but not necessarily as clearly right-skewed as the popularity metrics.

We note that to put the popularity distributions on a similar scale, we can use the scale factors below:

In [ ]:
view_scale_factor = 0.01
favorite_scale_factor = 1.0
comment_scale_factor = 10.0

Finally we can divide by Author Watcher count to normalize popularity based on how large a following an author already has (and thus has contributed to the popularity of the artwork):

In [ ]:
popularity_metrics = []
for n in range(df.shape[0]):
    view_count = df['Views'][n]
    favorite_count = df['Favorites'][n]
    comment_count = df['Comments'][n]
    watcher_count = df['Author Watchers'][n]
    popularity_metric = (view_count*view_scale_factor + favorite_count*favorite_scale_factor + comment_count*comment_scale_factor) / watcher_count
    popularity_metrics.append(popularity_metric * 10)

In [ ]:
popularity_metric_df = pd.DataFrame(popularity_metrics).rename(columns={0: 'Popularity Metric'})
popularity_metric_df.describe()

In [ ]:
df = pd.concat([df, popularity_metric_df], axis=1)

In [ ]:
sns.histplot(popularity_metrics)
plt.xlim(0, 200)
plt.show()

We still have some pretty wild outliers. Let's try applying a logarithm to the data to normalize further:

In [ ]:
df['LogPopMetric'] = np.log1p(df['Popularity Metric'])

In [ ]:
sns.histplot(df['LogPopMetric'])
plt.xlim(0, 10)
plt.show()

That seriously helped. Now that we've made a best attempt at normalizing our popularity data, we want to convert it into rank statistics, as popularity data is inherently ordinal in nature if we're sampling starting from the most popular. We'll use scipy's rankdata and append that to our dataframe:

In [ ]:
# flip ranks so we get the highest metric = rank 1
ranks = len(df) + 1 - rankdata(df['LogPopMetric'])
df['Rank'] = ranks

In [ ]:
df.head(3);

Just as a curiosity, let's order by ranking and see some of the most popular (as defined by us, of course) works:

In [ ]:
sorted_df = df.sort_values('Rank').reset_index(drop=True)
sorted_df.head(4);

In [ ]:
# deviation with the highest popularity metric
most_popular_id = sorted_df.loc[0][0]
most_popular_index = sorted_df[sorted_df['Deviation ID'] == f'{most_popular_id}'].index[0]
print(sorted_df[sorted_df['Deviation ID'] == f'{most_popular_id}'].loc[most_popular_index][10]);
display(Image(filename=f'./images/{most_popular_id}.png', width=300));

In [ ]:
# deviation with the fourth highest popularity metric
# ...choosing the fourth since second and third place are both also "stamp" memes like the most popular
second_most_popular_id = sorted_df.loc[3][0]
second_most_popular_index = sorted_df[sorted_df['Deviation ID'] == f'{second_most_popular_id}'].index[0]
print(sorted_df[sorted_df['Deviation ID'] == f'{second_most_popular_id}'].loc[second_most_popular_index][10]);
display(Image(filename=f'./images/{second_most_popular_id}.png', width=400));

In order to observe the data and to see the effects of keywords on popularity, we decided to aggregate all of the popularity data into a single dataframe with only the popularity values paired with keywords. In order to determine the popularity of a keyword, we decided to take the average of the popularities of each keyword and use that value to determine the overall popularity. We also removed all words that appeared less than 3 times in order to remove some inconsistent words that had high popularity rankings that appeared very few times. In order to do this, we used the values from the dictionary unique to create the new dataframe. We also did a log on the data in order to further deal with outliers.

First let's store the keywords and popularity statistics into one dictionary:

In [ ]:
# unique is a dict that contains a list of tuples of (Deviation ID, Popularity Metric, Rank) for each unique word in our df.
# the frequency of a key (word) is the length.
unique = {}
for n in range(df.shape[0]):
    tokens = word_tokenize(df['Caption'][n])
    for word in tokens:
        if word in unique:
            unique[word].append((df['Deviation ID'][n], df['LogPopMetric'][n], df['Rank'][n]))
        else:
            # tuple that contains (freq, deviation ID)
            unique[word] = [(df['Deviation ID'][n], df['LogPopMetric'][n], df['Rank'][n])]

In [ ]:
# for example:
unique['branch'];

Let's find a way to get the overall popularity of a word in another dictionary, "word_popularity". For the raw popularity metric we will take an average of a unique word's associated deviation (log) popularity list. Rank has no such issues, so we'll just average it.

In [ ]:
word_popularity = {}
for word in unique:
    pop_met_sum = 0
    rank_sum = 0
    for popular_tuple in unique[word]:
        deviation_id, pop_met, rank = popular_tuple
        pop_met_sum += pop_met
        rank_sum += rank
    total_unique = len(unique[word])
    pop_met_sum /= total_unique
    rank_sum /= total_unique
    word_popularity[word] = (pop_met_sum, rank_sum)

In [ ]:
# for example:
word_popularity['branch'];

Let's look at a plot of what we have before continuing.

In [ ]:
# a histplot that plots the frequency of avg logpopmet values for our words
logpopmet = [val[0] for val in word_popularity.values()]
sns.histplot(logpopmet)

We notice that... [data is still ordinal]

In [ ]:
# a histplot that plots the frequency of avg ranks for our words
ranks_for_hist = [val[1] for val in word_popularity.values()]
sns.histplot(ranks_for_hist)

We notice that when analyzing our data by rank, it follows a more Gaussian distribution. This is perhaps to be expected, since when data is ordinalized, it's probably more likely that a word will cover a group of deviations that have a more average rank to begin with.

Let's look at the spread and variance of our data to see if words seem to accurately predict popularity. In other words, let's see if the range and standard deviation of popularity for a single word is significantly smaller than the range and standard deviation overall.

In [ ]:
# here is the overall stdev and range
print('standard deviation is', np.std(df['LogPopMetric']))
print('range is', np.ptp(df['LogPopMetric']))

In [ ]:
count = 0
n = 5 # only analyze words that are related to >= n posts to get meaningful
logpop_stats = {}
rank_stats = {}
random_std_dev = [];

super_set = []
for word in unique:
    if len(unique[word]) >= n:
        count += 1
        logpop_data = [val[1] for val in unique[word]]
        logpop_stats[word] = (np.std(logpop_data), np.ptp(logpop_data))
        
        rank_data = [val[2] for val in unique[word]]
        rank_stats[word] = (np.std(rank_data), np.ptp(rank_data))
        
        super_set.extend(unique[word])
        #old implementation, accessed all deviations
        #for i in range(10):
        #    random_std_dev.append(np.std(df.sample(n=len(unique[word]))['LogPopMetric']))

#only use deviations our word based would have access to.
#note: it's literally a difference of 1 1187 vs 1188.
super_set = list(set(super_set))
for word in unique:
    if len(unique[word]) >= n:
        #we generate a lot to smooth out the randomness for consistency.
        for i in range(10):
            random_subset = random.sample(super_set, len(unique[word]))
            random_data = [val[1] for val in random_subset]
            random_std_dev.append(np.std(random_data))
print(count)

In [ ]:
np.std(df['LogPopMetric'])

In [ ]:
logpop_std_stats = [val[0] for val in logpop_stats.values()]
print(np.mean(logpop_std_stats))

sns.histplot(logpop_std_stats)
plt.xlabel('Standard Deviation')
plt.ylabel('Values that have a certain SD')

This is our distribution of standard deviations. Standard deviations here indicate how volatile a term is in popularity, and if terms directly correlated to popularity, we would expect all the terms to have a very low standard deviation. Our graph shows us that the standard deviation looks somewhat like a normal distribution, but that there is also a right skew. A right skew is what we expected, since our log popularity metric data has a right skew already and we may assume that our words are sampling that data (e.g. if we sampled 5 random words how would the standard deviation for that look?).

In [ ]:
sns.histplot(random_std_dev)
plt.xlabel('Standard Deviation')
plt.ylabel('Values that have a certain SD')

As seen above, the distribution of randomly sampled values looks normal, but in order to really compare we can use the t test to compare the populations.

In [ ]:
t_statistic, p_value = stats.ttest_ind(logpop_std_stats, random_std_dev)
print("P value:", p_value)

The P value gotten from comparing a random sampling of deviations' standard deviations and our standard deviations per word is usually/consistently low enough to reject the null hypothesis that the subset by word and by random samples are identical. We use this to assert that the word bears a significant enough influence on the popularity of posts containing words that a sample of the standard distribution of these differs from random. This supports our hypothesis that the caption influences the popularity of a post.

## Machine Learning
In order to more rigorously answer our hypothesis, we will use a predictive model using our data and see if it predicts better than simply guessing a mean.

In [ ]:
# setting up vectorizers
tfidf = TfidfVectorizer(sublinear_tf=True, 
                        analyzer='word', 
                        tokenizer=word_tokenize, 
                        stop_words='english')

count = CountVectorizer(analyzer='word',
                        tokenizer=word_tokenize,
                        stop_words='english')

# scramble the dataframe so that we don't have any bias based on ordering
scrambled_df = df.sample(frac=1.0)
training_df = df

training_tfidf = pd.DataFrame(tfidf.fit_transform(training_df['Caption'].tolist()).toarray(), 
                             index=training_df['Deviation ID'], 
                             columns=tfidf.get_feature_names_out())
training_count = pd.DataFrame(count.fit_transform(training_df['Caption'].tolist()).toarray(),
                             index=training_df['Deviation ID'],
                             columns=count.get_feature_names_out())

# setting up two decision tree regressors
tfidfTree = DecisionTreeRegressor()
countTree = DecisionTreeRegressor()

# setting up Lasso and SGD models better suited for sparse space
tfidfLasso = Lasso()
countLasso = Lasso()

# tfidfSGD = SGDRegressor()
# countSGD = SGDRegressor()

# setting a linear regression model
linearModel = LinearRegression()

In [ ]:
# this section was written with the help of ChatGPT
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import numpy as np

# Define the input features and output target
X = training_tfidf
y = training_df['Popularity Metric']

# Perform k-fold cross-validation and calculate RMSE
k = 10  # Number of folds
scoring = 'neg_root_mean_squared_error'

# tree models
tfidfTree_scores = cross_val_score(tfidfTree, training_tfidf, y, cv=k, scoring=scoring)
countTree_scores = cross_val_score(countTree, training_count, y, cv=k, scoring=scoring)

# lasso and SGDmodels
tfidfLasso_scores = cross_val_score(tfidfLasso, training_tfidf, y, cv=k, scoring=scoring)
countLasso_scores = cross_val_score(countLasso, training_count, y, cv=k, scoring=scoring)

# linear model
linear_scores = cross_val_score(linearModel, training_tfidf, y, cv=k, scoring=scoring)

# Define the K-Fold cross-validation object
kf = KFold(n_splits=k)

# Perform k-fold cross-validation for the mean baseline
mean_baseline_scores = []
for train_index, test_index in kf.split(X):
    y_train = y[train_index]
    y_test = y[test_index]
    mean_baseline_score = mean_squared_error(y_test, np.full_like(y_test, y_train.mean())) ** 0.5
    mean_baseline_scores.append(mean_baseline_score)

# Calculate the mean baseline score across all folds
mean_baseline_score_avg = np.mean(mean_baseline_scores)

# Calculate the average RMSE across all folds
print("Negative Root Mean Squared Error:")
print("Tree TF-IDF: ", -np.mean(tfidfTree_scores))
print("Tree Count: ", -np.mean(countTree_scores))
print("Lasso TF-IDF: ", -np.mean(tfidfLasso_scores))
print("Lasso Count: ", -np.mean(countLasso_scores))
print("Linear TF-IDF: ", -np.mean(linear_scores))
print("Mean Baseline: ", mean_baseline_score_avg)

Here we tried a multitude of models to see if we could achieve performance predicting the popularity better than a baseline of always guessing the mean. We did this using K-fold cross validation as there was high variance between models with different sections of the data.

Overall we found that Lasso stochastic coordinate descent with count vectorized tokens generally performed the best, and was able to sometimes slightly outperform the mean baseline. Being able to surpass the mean baseline means that our machine learning attempts do support our hypothesis, but likely not in a significant way since we only barely surpass the mean baseline possibly due to random chance, since it varies from performing slightly better to slightly worse.

In [ ]:
print("Shape of vectorized tokens: ", training_count.shape)
print("\nPerformance of Count Lasso models: ")
print(-countLasso_scores)
print("\nPerformance of Always Choose Mean: ")
print(mean_baseline_scores)

Here we take a look at the shape of our model and the performance of the models across K-fold. We can see that there is high variance in the standard loss, varying from as high as 711.167 to as low as 68.014 for the Count Lasso model, and that it's able to sometimes eke out a slight advantage over the mean baseline.

The shape of our data is also problematic, as the number of rows is almost equal to the number of columns, which means we'd likely have to see a direct correlation for the model to improve performance significantly. 

Given the shape of our data, and the high variance of the performance, we conclude that we do not have sufficient data for machine learning attempts. Potentially, with feature engineering we could increase performance.

In [ ]:
# analyzing lasso model by count vectorization features
lasso_model = Lasso()
lasso_model.fit(training_count, training_df['Popularity Metric'])

feature_importances = dict(zip(lasso_model.feature_names_in_, lasso_model.coef_))

sorted_tfidf_importances = sorted(feature_importances.items(), key=lambda x: x[1], reverse=True)
print('10 most popular features: ')
for a in sorted_tfidf_importances[:10]:
    print(a)
print('\n10 least popular features: ')
for a in reversed(sorted_tfidf_importances[-10:]):
    print(a)

We can look at the features and their importance of a lasso model that we fit to the entire dataset. Here we can see that a lot of the outliers are represented as very important within our model. It's likely that the model's minute success isn't predicated on these high importance outliers, but on some combination (not necessarily all) of the other roughly 1100 features, and that within our k-fold validation data these terms simply don't show up enough to express a worse result due to overfitting.

As an example, taking all the values with 4 digit coefficients, we can see they only show up once within the data set:

In [ ]:
print('Pluto:', unique['pluto'], '\nStamp:', unique['stamp'], '\nMusic:', unique['music'], '\nChristmas:', unique['christmas'])

In [ ]:
training_df = df
training_df = training_df.reset_index()

training_count = pd.DataFrame(count.fit_transform(training_df['Caption'].tolist()).toarray(),
                             columns=count.get_feature_names_out())

X = training_count.reset_index()
X = X.iloc[:, 1:]
X = sm.add_constant(X)
X, y = X.align(training_df['LogPopMetric'], axis=0)

statsmodel = sm.OLS(y, X)
result = statsmodel.fit()

print(result.f_pvalue)

Lastly, we ran ran an OLS regression model for inference on our vectorized input and popularity rating output, and received an incredibly small p-value for the F-test, which implies that having the independent variables (that is, the vectorized tokens) makes the model perform better than if there were no independent variables.

This strongly rejects the null hypothesis that subject as described by CLIP has no bearing on popularity, which supports our hypothesis that the subject as described by CLIP does, to some extent, determine popularity. This helps to validate the previous machine learning analysis.

It's important to note that statsmodel's OLS (ordinary least squares) model may not be the model best suited for analysis of sparse data, and does tend to vary likely due to the aforementioned problems with our data.

# Ethics & Privacy

There are a number of ethical concerns regarding this research question that we must be mindful of as we analyze data. The most obvious issue is that we are tagging artwork as unpopular by virtue of not identifying said artwork as popular. However, this should not be a strong issue as we are not presenting identifying pieces of information of specific pieces of artwork or individual artists, so it should not be possible to label a specific artwork or artist as “unpopular”.

In terms of normalization, a possible solution would be to take a ratio between the number of likes on the artwork and the number of followers that certain artist has in order to take into account the disparity between larger artists and smaller artists in terms of popularity, as more popular artists would get more likes due to a larger audience. Additionally, it is entirely possible that our analysis may exclude cultural influences of minority groups. Since those residing in developed countries have more leisure time/resources (such as drawing software or drawing e-tablets), it is plausible that most digital art posted to social media is likely from developed countries. Thus, the work we analyze may disproportionately represent artwork and cultural trends of majority groups of developed countries while glossing over minority groups, which tend to be similar across developed countries.

Finally, because the artworks are on a public forum, they have consented to allowing their art to be analyzed. The Deviantart TOS states that you cannot “reproduce, distribute, publicly display or perform, or prepare derivative works”, which does not include the use of the artworks for an analytic survey. Although there is no clear-cut solution for this, it serves us well to keep this fact in mind when drawing conclusions upon our analyses.

# Conclusion & Discussion

In our project, we aimed to assess whether the features and subject of an artwork identified by CLIP were a significant factor in the popularity of the post. We started by scraping 1188 images off of deviantart, done by traversing the ‘popular’ page continuously and then captioning them using Automatic 1111’s Stable Diffusion WebUI’s API. 

After cleaning our data and viewing it in a number of histograms, we attempted to ascertain the importance of words to predicting popularity. We initially did this by going over our words, gathering the subset of deviations a word was featured in, and taking the standard deviation. We made sure that these subsets had 5 or more observations in them. At the same time, we generated another proportional but larger (to deal with the variability of randomness) set of subsets that randomly sampled our data. By running a t-test on the two populations we found that consistently the p value was less than 0.05, passing our significance test. This supported our hypothesis, showing that the words found in the caption of a post had a statistically significant impact on how different in popularity it would be from other similar posts. 

To attempt to investigate the significance of the features in the caption, we attempted to train machine learning models on the caption and popularity metric, but found that the models generally performed worse than a mean baseline. One model, a lasso coordinate descent model, however, did beat the mean baseline by a tiny margin. Due to the inability of our models to perform, we were unable to verify that the subject features of a post significantly predicts its popularity. The last model we looked at was an ordinary least squares model, which allowed us to infer the impact of the independent variables (the features) on the dependent variable (the popularity). This model confirmed what we already knew from our previous data analysis, but was highly variable.

A consideration that we could have taken into account is standardizing the type of picture by choosing specific images that have similar features or contain the same tags. This way, the pictures would have some sort of similarity between one another instead of using any picture pulled from DeviantArt. With a similarity, we could have found more accurately specific features that would have influenced the relative popularity of the drawing since there would have been some normalization of the images that were used in the sample, and the differentiating features would have been more influential to the popularity.

Another limitation to this project was the amount of outliers present in the data scraped from DeviantArt. Though we expected that the data obtained using the DeviantArt API would have some outliers, the amount that was **actually** received from the scraping tool was higher than expected. As a result, the outliers interfered with our prediction models. Though we employed a wide range of predictors, such as Decision Tree Regressors, OLS, and Lasso Models, they were **all** impacted by the outliers. The top 2 terms impacting our models were ‘pluto’ and ‘stamp’ of 4 total *greater than 1000* coefficients, the most popular post of course being a stamp with a picture of pluto. 

Lastly, the amount of images scraped from DeviantArt also affected the performance of the prediction models, and project overall, as it was not a large enough number to account for the wide breadth of features found in our dataset. To improve this model, we would scrape much more data in an attempt to support our hypothesis in that features have a **significant** effect, as so far we have seen that the features have a *statistically* significant effect only.

# Team Contributions

**Mariam**
- Helped create the timeline for meetings and deadlines as outlined in project proposal
- Helped to interpret and write descriptions for statistic graphs for the EDA
- Contributed to research for project background
- Wrote and edited part of script for project video

**Alexandra** 
- Prepared/initialized the creation of the notebooks (outside of the template)
- Completed the project background for the Project Proposal and its proofreadings
- Wrote and edited part of script for project video
- Did descriptions for statistic graphs for the EDA

**Brian**
- Wrote the ethics and privacy portion of the Project Proposal
- Contributed to data analysis with keyword popularity statistics
- Helped with the video script and video creation
- Helped finalize the project document

**Andrew**
- Contributed to data extraction and generation
- Main work on data cleaning, making dataset ethically useable and applied natural language processing on AI generated captions
- Worked on construction of popularity metric
- Applied statistical analysis relevant to hypothesis testing

**Ethan**
- Contributed to data extraction and generation
- Main work on data extraction, scraping relevant data from Deviantart and processing the images to create captions
- Mostly worked on the data portion of the project proposal
- Mostly worked on the ML portion of the EDA